# CNN architicture and training

## Mount Google Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Libraries

In [8]:
import sys
import os

from sklearn.utils import class_weight

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D
from tensorflow.keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import Model
from keras import models
from tensorflow.keras.utils import to_categorical
import time
import matplotlib.pyplot as plt
from tensorflow.keras.utils import plot_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
from tensorflow.keras.models import load_model
from tqdm import trange
import tensorflow as tf
import pandas as pd
import seaborn as sns
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
import cv2
from keras.preprocessing import image
from os import listdir
from tensorflow.keras.optimizers import SGD
from os.path import isfile, join
from tensorflow.keras import layers

## Inputs

In [11]:
# Access VCNN4PuDe Folder
%cd '/content/drive/My Drive/VCNN4PuDe'
train_data_dir="path of the training set/"
validation_data_dir="path of validation set/"


/content/drive/My Drive/VCNN4PuDe


## CNN architicture

In [7]:
# Other models MobileNet, EfficientNetV2B0, Xception, DenseNet121, ResNet50
from tensorflow.keras.applications import EfficientNetB0

def get_model():

  img_rows, img_cols = 224,224
  model = Sequential()

  model = EfficientNetB0(weights='imagenet', include_top = False,  input_shape = (img_rows, img_cols, 3))

  for layer in model.layers:
    layer.trainable = True


  top_model =model.output
  top_model = GlobalAveragePooling2D()(top_model)
  output = Dense(1,activation='sigmoid')(top_model)

  model = Model(inputs = model.input, outputs = output)

  model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

  return model

model = get_model()
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling_4 (Rescaling)        (None, 224, 224, 3)  0           ['input_3[0][0]']                
                                                                                                  
 normalization_2 (Normalization  (None, 224, 224, 3)  7          ['rescaling_4[0][0]']            
 )                                                                                                
                                                                                            

## Loading the training and test sets

In [ ]:
train_datagen = ImageDataGenerator()
validation_datagen = ImageDataGenerator()

epochs=100
batch_size = 32
img_rows, img_cols=224,224
train_generator = train_datagen.flow_from_directory(
          train_data_dir,
          target_size=(img_rows, img_cols),
          batch_size=batch_size,

           class_mode='binary')


validation_generator = validation_datagen.flow_from_directory(
          validation_data_dir,
          target_size=(img_rows, img_cols),
          batch_size=batch_size,
           class_mode='binary')

## Training

In [ ]:
filepath="Name of the trained mode"+'-epoch{epoch:02d}-ACC{val_accuracy:.2f}.h5'

checkpoint = ModelCheckpoint(filepath ,
                             monitor="val_accuracy",
                             mode="max",
                             save_best_only = True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_accuracy',
                          min_delta = 0,
                          patience = 20,
                          verbose = 1,
                          restore_best_weights = True
                          )
callbacks = [earlystop, checkpoint]

  # Fit data to model
history = model.fit(
    train_generator,
    epochs = epochs,
    callbacks = callbacks,
    batch_size =batch_size,
    validation_data = validation_generator,

    )
